# Group $\mathsf{G}_3$

Given a combinatorial type $\mathsf{w}$, let $\Sigma_{L}$ be the subcomplex of $\mathsf{TS}(\mathsf{w})$ obtained by removing each leaf-pair. The combinatorial types $\mathsf{w}$ belonging to $\mathsf{G}_3$ are those not in $\mathsf{G}_i$ ($i\leq 2$), and such that $\Sigma_{L}$ is vertex-connecting. 

In [ ]:
using Oscar
using Combinatorics
pm = Polymake

In [ ]:
cd("..")

In [ ]:
currentDir = pwd()
include(joinpath(currentDir, "src/inputData38.jl"));
include(joinpath(currentDir, "src/fileHandling.jl"));
include(joinpath(currentDir, "src/tscCoordRing.jl"));
include(joinpath(currentDir, "src/matroidalSubd.jl"));
include(joinpath(currentDir, "src/Bmaximal.jl"));
include(joinpath(currentDir, "src/simplifyIdeal.jl"));

The list ```G3``` contains all combinatorial types in $\mathsf{G}_3$. 

In [ ]:
G3Path = joinpath(currentDir,"groupsFinal/G3.dat")
G3 = file2SetVectors(G3Path);

For a given combinatorial type $\mathsf{w}$, we determine $\Sigma_L$, and compute its coordinate ring using Proposition 6.3, implemented by the function ```limitTSC```. Then we determine those whose ideal $I_{\Sigma_L}$ satisfies the hypotheses of Lemma 6.11. 

**Warning!! The cell below takes a long time to run.**

Better to run on a small sample size. To handle the entire loop, run in a terminal, using ```julia```, from the main directory, the file ```G3Data/G3HasUpperTriangular.jl```. This takes ~8.5 hours. 

In [ ]:
hasUpperTriangularG3 = []
doesntHaveUpperTriangularG3 = []

R, x = makePolyRing(3,8, QQ)

UTFile = joinpath(currentDir,"G3Data/hasUpperTriangularG3.dat")
NUTFile = joinpath(currentDir,"G3Data/doesnthaveUpperTriangularG3.dat")

io1 = open(UTFile, "w") 
io2 = open(NUTFile, "w")
close(io1)
close(io2)


for w in G3
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    Ms = subdMatroidsNonLeaves(subd,3,8)
    
    optBs = optimalBasesForLimit(Ms)
    Lim = limitTSC(Ms, QQ, first(optBs), R, x)
    
    Rt = base_ring(Lim)
    xt = gens(base_ring(Lim))
    gLim = minimal_generating_set(Lim)
    
    if systemHasUniUpperTriangle(gLim, Rt, xt)
        push!(hasUpperTriangularG3, w)

        open(UTFile, "a") do io
            write(io, vec2String(w), "\n")
        end;

        
    else
        push!(doesntHaveUpperTriangularG3, w)
        open(NUTFile, "a") do io
            write(io, vec2String(w), "\n")
        end;
    end
    
end

In [ ]:
hasUpperTriangularG3 = file2SetVectors(joinpath(currentDir, "G3Data/hasUpperTriangularG3-precomputed.dat"));
doesntHaveUpperTriangularG3 = file2SetVectors(joinpath(currentDir, "G3Data/doesnthaveUpperTriangularG3-precomputed.dat"));

We appply Algorithm 6.12 to the combinatorial types in ```doesntHaveUpperTriangularG3```. If the output ideal is $\langle 0 \rangle$, then the function ```canReduceIdeal``` returns ```true```. In this case, the ring $R_{\Sigma_{L}}$ is a regular integral domain. 

**Warning!! The cell below takes a long time to run.**

Better to run on a small sample size. To handle the entire loop, run in a terminal, using ```julia```, from the main directory, the file ```G3Data/G3SimplifyIdeals.jl```. This takes ~40 min.

In [ ]:
canFullySimplifyG3 = []
cannotFullySimplifyG3 = []

R, x = makePolyRing(3,8, QQ)

canFile = joinpath(currentDir,"G3Data/canFullySimplifyG3.dat")
cannotFile = joinpath(currentDir,"G3Data/cannotFullySimplifyG3.dat")

#io1 = open(canFile, "w") 
#io2 = open(cannotFile, "w")
#close(io1)
#close(io2)


for w in doesntHaveUpperTriangularG3
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    Ms = subdMatroidsNonLeaves(subd,3,8)
    
    optBs = optimalBasesForLimit(Ms)
    Lim = limitTSC(Ms, QQ, first(optBs), R, x)
    gLim = minimal_generating_set(Lim)
    
    Sinv = localizingSemiGroup(Ms, QQ, first(optBs), R, x)
    SR = Localization(Sinv)[1]
    
    if canReduceIdeal(gLim, R, x, Sinv, SR)
        push!(canFullySimplifyG3, w)
        
        open(canFile, "a") do io
           write(io, vec2String(w), "\n")
        end;
    else
        push!(cannotFullySimplifyG3, w)
        
        open(cannotFile, "a") do io
           write(io, vec2String(w), "\n")
        end;
    end
end



As we see, ```canReduceIdeal``` returns true for all of the combinatorial types in ```doesntHaveUpperTriangularG3```.

In [ ]:
cannotFullySimplifyG3 = file2SetVectors(joinpath(currentDir, "G3Data/cannotFullySimplifyG3-precomputed.dat"));
print(length(cannotFullySimplifyG3) == 0)

## Dimension of $\operatorname{Gr}(\mathsf{w})$

Next we check that the dimensions $\operatorname{Gr}(\mathsf{w})$ are 15 for each $\mathsf{w}$ in $\mathsf{G}_3$. We do this by applying Proposition 6.9, which is implemented in the function ```dimLimitLeafCVP``` below. 

---
**Function**: ```dimBRing_MatList(Ms::Vector{Matroid}, xi::Vector{Int64})```

*Description*: 

*Example*: 

---
**Function**: ```dimLimitTSC(Ms::Vector{Matroid}, F::Ring, B::Vector{Int64}, R::GradedPolynomialRing, x::fmpz_mpoly)```

*Description*: 

*Example*: 

---
**Function**: ```dimLimitLeafCVP(subd::SubdivisionOfPoints, F::Ring, d::Int64, n::Int64, R::GradedPolynomialRing, x::fmpz_mpoly)```

*Description*: 

*Example*: 


In [ ]:
function dimLimitLeafCVP(subd, F, d, n, R, x)
    (leafMaxMatroids, leafEdgeMaxMatroids, remainingMaxMatroids)  = subd2LeavesAndCenter(subd, d, n)
    
    dimsLeafMax = [dimTSC_Optimized(M, F, R, x) for M in leafMaxMatroids ]; 
    dimsLeafEdge = [dimTSC_Optimized(M, F, R, x) for M in leafEdgeMaxMatroids]
    dimCenter = dimLimitTSC(remainingMaxMatroids, F, R, x)
    
    return sum(dimsLeafMax) - sum(dimsLeafEdge) + dimCenter
end

**Warning!! The cell below takes a long time to run.**

Better to run on a small sample size. To handle the entire loop, run in a terminal, using ```julia```, from the main directory, the file ```G3Data/G3Dims.jl```. This takes ~9 hours.

In [ ]:
R, x =  makePolyRing(3,8,QQ)
dims_G3 = []


dimFile = joinpath(currentDir, "G3Data/dims_G3.dat")
io1 = open(dimFile, "w") 
close(io1)


for i in 1:10
    w = G3[i]
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w)
    d = dimLimitLeafCVP(subd, QQ, 3, 8, R, x)
    push!(dims_G3, d)
    
    open(dimFile, "a") do io
        write(io, string("dim w", i, " = " , d, "\n"))
        end
    
end
